In [1]:
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json

## Define the variables

In [5]:
hosts = ["127.0.0.1"]
file_path = './data/full.csv'
file_path_json = './data/full.json'
index_name = 'anpr-index'
doc_type = 'comuni'

### Transform the csv into a json

In [6]:
df = pd.read_csv(file_path, sep=";" )
df.to_json(file_path_json, orient='records')

In [7]:
with open(file_path_json, 'r') as f:
    data = json.load(f)

In [8]:
data[0]

{'ID': 12560,
 'DATAISTITUZIONE': '1866-11-19',
 'DATACESSAZIONE': '13/11/1924',
 'CODISTAT': 28001,
 'CODCATASTALE': 'A001',
 'DENOMINAZIONE_IT': 'ABANO',
 'DENOMTRASLITTERATA': 'ABANO',
 'ALTRADENOMINAZIONE': None,
 'ALTRADENOMTRASLITTERATA': None,
 'IDPROVINCIA': 28,
 'IDREGIONE': 5,
 'IDPREFETTURA': None,
 'STATO': 'C',
 'SIGLAPROVINCIA': 'PD',
 'FONTE': None,
 'DATAULTIMOAGG': '2016-06-17',
 'COD_DENOM': 28500.0,
 'ATTIVO': False,
 'Codice Città Metropolitana': '-',
 'Codice Ripartizione Geografica': 2.0,
 'Ripartizione geografica': 'Nord-est',
 'Denominazione regione': 'Veneto',
 'Denominazione Città metropolitana': '-',
 'Denominazione provincia': 'Padova',
 'Flag Comune capoluogo di provincia': 0.0,
 'Sigla automobilistica': 'PD',
 'Codice Comune formato numerico': 28001.0,
 'Codice Comune numerico con 110 province (dal 2010 al 2016)': 28001.0,
 'Codice Comune numerico con 107 province (dal 2006 al 2009)': 28001.0,
 'Codice Comune numerico con 103 province (dal 1995 al 2005)': 

## Open the connection to Elastic and index the documents

In [9]:
es = Elasticsearch(hosts=hosts)

In [10]:
if not es.indices.exists(index_name):
    es.indices.create(index_name)

In [11]:
es.indices.get(index_name)

{'anpr-index': {'aliases': {},
  'mappings': {},
  'settings': {'index': {'creation_date': '1528714408780',
    'number_of_shards': '5',
    'number_of_replicas': '1',
    'uuid': 'Ce-uRlwFR728P4w3G8h1yQ',
    'version': {'created': '5060999'},
    'provided_name': 'anpr-index'}}}}

In [12]:
def create_req(id_doc, doc, op='index'):
    return {
        '_op_type': op,
        '_index': index_name,
        '_type': doc_type,
        '_id': id_doc,
        'doc': doc
    }

In [13]:
operations = (create_req(d['CODISTAT'], d) for d in data)

In [14]:
results = helpers.bulk(es, operations)
results

(12113, [])